In [1]:
%matplotlib qt

import matplotlib.pyplot as plt

plt.plot([0,1,2], [0, 1, 4])
plt.show()

In [3]:
# https://matplotlib.org/3.1.1/users/event_handling.html

import numpy as np

fig, ax = plt.subplots()
ax.plot(np.random.rand(10))

def onclick(event):
    print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          ('double' if event.dblclick else 'single', event.button,
           event.x, event.y, event.xdata, event.ydata))

cid = fig.canvas.mpl_connect('button_press_event', onclick)

single click: button=1, x=211, y=331, xdata=2.164718, ydata=0.712769
single click: button=1, x=156, y=234, xdata=1.066935, ydata=0.486923
single click: button=1, x=128, y=194, xdata=0.508065, ydata=0.393790
single click: button=1, x=302, y=406, xdata=3.981048, ydata=0.887392
single click: button=1, x=380, y=377, xdata=5.537903, ydata=0.819871
single click: button=1, x=185, y=245, xdata=1.645766, ydata=0.512534
single click: button=1, x=252, y=157, xdata=2.983065, ydata=0.307643
single click: button=1, x=204, y=89, xdata=2.025000, ydata=0.149318
single click: button=1, x=133, y=95, xdata=0.607863, ydata=0.163288
single click: button=1, x=187, y=406, xdata=1.685685, ydata=0.887392
single click: button=1, x=200, y=90, xdata=1.945161, ydata=0.151646
single click: button=1, x=165, y=133, xdata=1.246573, ydata=0.251763
single click: button=1, x=166, y=200, xdata=1.266532, ydata=0.407760
single click: button=1, x=170, y=303, xdata=1.346371, ydata=0.647576
single click: button=1, x=170, y=340,

In [4]:
class LineBuilder:
    def __init__(self, line):
        self.line = line
        self.xs = list(line.get_xdata())
        self.ys = list(line.get_ydata())
        self.cid = line.figure.canvas.mpl_connect('button_press_event', self)

    def __call__(self, event):
        print('click', event)
        if event.inaxes!=self.line.axes: return
        self.xs.append(event.xdata)
        self.ys.append(event.ydata)
        self.line.set_data(self.xs, self.ys)
        self.line.figure.canvas.draw()

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title('click to build line segments')
line, = ax.plot([0], [0])  # empty line
linebuilder = LineBuilder(line)

plt.show()

click button_press_event: xy=(153, 283) xydata=(-0.03881048387096775, 0.013511904761904767) button=1 dblclick=False inaxes=AxesSubplot(0.125,0.11;0.775x0.77)
click button_press_event: xy=(271, 334) xydata=(-0.012641129032258072, 0.028690476190476197) button=1 dblclick=False inaxes=AxesSubplot(0.125,0.11;0.775x0.77)
click button_press_event: xy=(371, 330) xydata=(0.009536290322580646, 0.02750000000000001) button=1 dblclick=False inaxes=AxesSubplot(0.125,0.11;0.775x0.77)
click button_press_event: xy=(402, 277) xydata=(0.016411290322580638, 0.011726190476190487) button=1 dblclick=False inaxes=AxesSubplot(0.125,0.11;0.775x0.77)
click button_press_event: xy=(360, 213) xydata=(0.007096774193548386, -0.007321428571428576) button=1 dblclick=False inaxes=AxesSubplot(0.125,0.11;0.775x0.77)
click button_press_event: xy=(309, 180) xydata=(-0.004213709677419367, -0.01714285714285714) button=1 dblclick=False inaxes=AxesSubplot(0.125,0.11;0.775x0.77)
click button_press_event: xy=(242, 191) xydata=(-0

In [5]:
class DraggableRectangle:
    def __init__(self, rect):
        self.rect = rect
        self.press = None

    def connect(self):
        'connect to all the events we need'
        self.cidpress = self.rect.figure.canvas.mpl_connect(
            'button_press_event', self.on_press)
        self.cidrelease = self.rect.figure.canvas.mpl_connect(
            'button_release_event', self.on_release)
        self.cidmotion = self.rect.figure.canvas.mpl_connect(
            'motion_notify_event', self.on_motion)

    def on_press(self, event):
        'on button press we will see if the mouse is over us and store some data'
        if event.inaxes != self.rect.axes: return

        contains, attrd = self.rect.contains(event)
        if not contains: return
        print('event contains', self.rect.xy)
        x0, y0 = self.rect.xy
        self.press = x0, y0, event.xdata, event.ydata

    def on_motion(self, event):
        'on motion we will move the rect if the mouse is over us'
        if self.press is None: return
        if event.inaxes != self.rect.axes: return
        x0, y0, xpress, ypress = self.press
        dx = event.xdata - xpress
        dy = event.ydata - ypress
        #print('x0=%f, xpress=%f, event.xdata=%f, dx=%f, x0+dx=%f' %
        #      (x0, xpress, event.xdata, dx, x0+dx))
        self.rect.set_x(x0+dx)
        self.rect.set_y(y0+dy)

        self.rect.figure.canvas.draw()


    def on_release(self, event):
        'on release we reset the press data'
        self.press = None
        self.rect.figure.canvas.draw()

    def disconnect(self):
        'disconnect all the stored connection ids'
        self.rect.figure.canvas.mpl_disconnect(self.cidpress)
        self.rect.figure.canvas.mpl_disconnect(self.cidrelease)
        self.rect.figure.canvas.mpl_disconnect(self.cidmotion)

fig = plt.figure()
ax = fig.add_subplot(111)
rects = ax.bar(range(10), 20*np.random.rand(10))
drs = []
for rect in rects:
    dr = DraggableRectangle(rect)
    dr.connect()
    drs.append(dr)

plt.show()

event contains (2.6, 0)
event contains (3.6, 0)
event contains (4.6, 0)
event contains (5.6, 0)
event contains (6.6, 0)
event contains (7.6, 0)
event contains (8.6, 0)


In [6]:
class DraggableRectangle:
    lock = None  # only one can be animated at a time
    def __init__(self, rect):
        self.rect = rect
        self.press = None
        self.background = None

    def connect(self):
        'connect to all the events we need'
        self.cidpress = self.rect.figure.canvas.mpl_connect(
            'button_press_event', self.on_press)
        self.cidrelease = self.rect.figure.canvas.mpl_connect(
            'button_release_event', self.on_release)
        self.cidmotion = self.rect.figure.canvas.mpl_connect(
            'motion_notify_event', self.on_motion)

    def on_press(self, event):
        'on button press we will see if the mouse is over us and store some data'
        if event.inaxes != self.rect.axes: return
        if DraggableRectangle.lock is not None: return
        contains, attrd = self.rect.contains(event)
        if not contains: return
        print('event contains', self.rect.xy)
        x0, y0 = self.rect.xy
        self.press = x0, y0, event.xdata, event.ydata
        DraggableRectangle.lock = self

        # draw everything but the selected rectangle and store the pixel buffer
        canvas = self.rect.figure.canvas
        axes = self.rect.axes
        self.rect.set_animated(True)
        canvas.draw()
        self.background = canvas.copy_from_bbox(self.rect.axes.bbox)

        # now redraw just the rectangle
        axes.draw_artist(self.rect)

        # and blit just the redrawn area
        canvas.blit(axes.bbox)

    def on_motion(self, event):
        'on motion we will move the rect if the mouse is over us'
        if DraggableRectangle.lock is not self:
            return
        if event.inaxes != self.rect.axes: return
        x0, y0, xpress, ypress = self.press
        dx = event.xdata - xpress
        dy = event.ydata - ypress
        self.rect.set_x(x0+dx)
        self.rect.set_y(y0+dy)

        canvas = self.rect.figure.canvas
        axes = self.rect.axes
        # restore the background region
        canvas.restore_region(self.background)

        # redraw just the current rectangle
        axes.draw_artist(self.rect)

        # blit just the redrawn area
        canvas.blit(axes.bbox)

    def on_release(self, event):
        'on release we reset the press data'
        if DraggableRectangle.lock is not self:
            return

        self.press = None
        DraggableRectangle.lock = None

        # turn off the rect animation property and reset the background
        self.rect.set_animated(False)
        self.background = None

        # redraw the full figure
        self.rect.figure.canvas.draw()

    def disconnect(self):
        'disconnect all the stored connection ids'
        self.rect.figure.canvas.mpl_disconnect(self.cidpress)
        self.rect.figure.canvas.mpl_disconnect(self.cidrelease)
        self.rect.figure.canvas.mpl_disconnect(self.cidmotion)

fig = plt.figure()
ax = fig.add_subplot(111)
rects = ax.bar(range(10), 20*np.random.rand(10))
drs = []
for rect in rects:
    dr = DraggableRectangle(rect)
    dr.connect()
    drs.append(dr)

plt.show()

event contains (3.6, 0)
event contains (3.6, 0)
event contains (2.6437096774193543, 2.325141059407885)
event contains (2.6437096774193543, 2.277689201052622)
event contains (2.6437096774193543, 2.277689201052622)
event contains (4.6, 0)
event contains (0.6, 0)
event contains (1.6, 0)
event contains (2.2742338709677417, 3.463985659934197)
event contains (3.0124596774193546, 1.6608150424342032)
event contains (6.6, 0)


In [7]:
def enter_axes(event):
    print('enter_axes', event.inaxes)
    event.inaxes.patch.set_facecolor('yellow')
    event.canvas.draw()

def leave_axes(event):
    print('leave_axes', event.inaxes)
    event.inaxes.patch.set_facecolor('white')
    event.canvas.draw()

def enter_figure(event):
    print('enter_figure', event.canvas.figure)
    event.canvas.figure.patch.set_facecolor('red')
    event.canvas.draw()

def leave_figure(event):
    print('leave_figure', event.canvas.figure)
    event.canvas.figure.patch.set_facecolor('grey')
    event.canvas.draw()

fig1 = plt.figure()
fig1.suptitle('mouse hover over figure or axes to trigger events')
ax1 = fig1.add_subplot(211)
ax2 = fig1.add_subplot(212)

fig1.canvas.mpl_connect('figure_enter_event', enter_figure)
fig1.canvas.mpl_connect('figure_leave_event', leave_figure)
fig1.canvas.mpl_connect('axes_enter_event', enter_axes)
fig1.canvas.mpl_connect('axes_leave_event', leave_axes)

fig2 = plt.figure()
fig2.suptitle('mouse hover over figure or axes to trigger events')
ax1 = fig2.add_subplot(211)
ax2 = fig2.add_subplot(212)

fig2.canvas.mpl_connect('figure_enter_event', enter_figure)
fig2.canvas.mpl_connect('figure_leave_event', leave_figure)
fig2.canvas.mpl_connect('axes_enter_event', enter_axes)
fig2.canvas.mpl_connect('axes_leave_event', leave_axes)

plt.show()


enter_axes AxesSubplot(0.125,0.53;0.775x0.35)
enter_figure Figure(640x480)
leave_figure Figure(640x480)
enter_figure Figure(640x480)
enter_axes AxesSubplot(0.125,0.53;0.775x0.35)
leave_axes AxesSubplot(0.125,0.53;0.775x0.35)
enter_axes AxesSubplot(0.125,0.53;0.775x0.35)
leave_axes AxesSubplot(0.125,0.53;0.775x0.35)
leave_figure Figure(640x480)
enter_axes AxesSubplot(0.125,0.53;0.775x0.35)
enter_figure Figure(640x480)
leave_axes AxesSubplot(0.125,0.53;0.775x0.35)
leave_figure Figure(640x480)
enter_axes AxesSubplot(0.125,0.53;0.775x0.35)
enter_figure Figure(640x480)
leave_axes AxesSubplot(0.125,0.53;0.775x0.35)
enter_axes AxesSubplot(0.125,0.11;0.775x0.35)
leave_axes AxesSubplot(0.125,0.11;0.775x0.35)
enter_axes AxesSubplot(0.125,0.53;0.775x0.35)
leave_axes AxesSubplot(0.125,0.53;0.775x0.35)
leave_figure Figure(640x480)
enter_figure Figure(640x480)
enter_axes AxesSubplot(0.125,0.53;0.775x0.35)
leave_axes AxesSubplot(0.125,0.53;0.775x0.35)
enter_axes AxesSubplot(0.125,0.11;0.775x0.35)
lea

In [8]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title('click on points')

line, = ax.plot(np.random.rand(100), 'o', picker=5)  # 5 points tolerance

def onpick(event):
    thisline = event.artist
    xdata = thisline.get_xdata()
    ydata = thisline.get_ydata()
    ind = event.ind
    points = tuple(zip(xdata[ind], ydata[ind]))
    print('onpick points:', points)

fig.canvas.mpl_connect('pick_event', onpick)

plt.show()

onpick points: ((54.0, 0.9111918415510254),)
onpick points: ((47.0, 0.7035420185507757),)
onpick points: ((73.0, 0.6162711889081424),)
onpick points: ((80.0, 0.6072872572983121),)
onpick points: ((74.0, 0.3683707360912535),)
onpick points: ((75.0, 0.3457112586865134),)


In [9]:
X = np.random.rand(100, 1000)
xs = np.mean(X, axis=1)
ys = np.std(X, axis=1)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title('click on point to plot time series')
line, = ax.plot(xs, ys, 'o', picker=5)  # 5 points tolerance


def onpick(event):

    if event.artist!=line: return True

    N = len(event.ind)
    if not N: return True


    figi = plt.figure()
    for subplotnum, dataind in enumerate(event.ind):
        ax = figi.add_subplot(N,1,subplotnum+1)
        ax.plot(X[dataind])
        ax.text(0.05, 0.9, 'mu=%1.3f\nsigma=%1.3f'%(xs[dataind], ys[dataind]),
                transform=ax.transAxes, va='top')
        ax.set_ylim(-0.5, 1.5)
    figi.show()
    return True

fig.canvas.mpl_connect('pick_event', onpick)

plt.show()